In [27]:
import json
import pandas as pd
import numpy as np
from copy import deepcopy
from IPython.display import display
import re

from modules.team import Team, BenchTeam
from modules.player import Player, Position
from modules.transfer import Transfer
from modules.fixture_difficulty_matrix import FixtureDifficultyMatrix
from modules.utils import getDataFilesSorted

import config

In [28]:
CURRENT_DATE = config.CURRENT_DATE
teams_filename = f"./results/{CURRENT_DATE}/results_{CURRENT_DATE}.json"
scores_filename = f"./results/{CURRENT_DATE}/scores_{CURRENT_DATE}.json"
SELECTED_MODEL_INDEX = 2
TOTAL_BUDGET = 1000

In [29]:
current_team_names = {"Jordan Pickford",
                      "André Onana",
                      "Nathan Collins",
                      "Riccardo Calafiori",
                      "Rayan Aït-Nouri",
                      "Fabian Schär",
                      "Daniel Ballard",
                      "Morgan Rogers",
                      "Bryan Mbeumo",
                      "Alex Iwobi",
                      "Jacob Murphy",
                      "Brennan Johnson",
                      "Yoane Wissa",
                      "Alexander Isak",
                      "Richarlison de Andrade",
                      }

In [30]:
# TODO: Change how files are loaded

with open(teams_filename, "r") as f:
    tempJson: list[dict] = json.load(f)

actualJson = tempJson[SELECTED_MODEL_INDEX]

In [31]:
all_player_data = pd.DataFrame.from_records(actualJson["players"])

In [32]:
print(all_player_data)

     cost  form  gameweek  ict_index   id                          name  \
0      55   8.0         2        6.2    1             David Raya Martín   
1      45   0.0         2        0.0    2    Kepa Arrizabalaga Revuelta   
2      40   0.0         2        0.0    3                     Karl Hein   
3      40   0.0         2        0.0    4                 Tommy Setford   
4      61   6.0         2        5.0    5  Gabriel dos Santos Magalhães   
..    ...   ...       ...        ...  ...                           ...   
702    40   0.0         2        0.0  703                   Sam Proctor   
703    45   0.0         2        0.0  704                 Tyler Onyango   
704    40   0.0         2        0.0  705               Elijah Campbell   
705    40   0.0         2        0.0  706                  James Justin   
706    55   0.0         2        0.0  707                 Tyler Dibling   

    opposing_team  points_last_week  points_per_game position  season  \
0             LEE         

In [33]:
currentTeamPlayers = all_player_data.loc[all_player_data["name"].isin(current_team_names)]


for player in current_team_names:
    if(player not in currentTeamPlayers["name"].values):
        raise ValueError(f"player '{player}' not found")

In [34]:
current_team = Team.fromDataFrame(currentTeamPlayers)

In [35]:
current_team_cost = current_team.getTotalCost()
current_team_cost

969

In [36]:
with open(teams_filename,"r",encoding="utf-8") as f:
    all_data = json.load(f)

all_data = all_data[SELECTED_MODEL_INDEX]
selected_team_df = pd.DataFrame.from_records(all_data["team"])

In [37]:
selected_team = Team.fromDataFrame(selected_team_df)

In [38]:
display(selected_team)

In [39]:
new_players = selected_team - current_team
print(new_players)


Total Score: 177.27

Goalkeepers:
- Guglielmo Vicario	Score: 17.65	Cost: 50	Fixture Difficulty: 0.0
Defenders:
- William Saliba	Score: 15.15	Cost: 61	Fixture Difficulty: 0.0
- Jurriën Timber (Captain) 	Score: 22.92	Cost: 56	Fixture Difficulty: 0.0
- Trevoh Chalobah	Score: 13.8	Cost: 50	Fixture Difficulty: 0.0
Attackers:
- Hugo Ekitiké	Score: 19.67	Cost: 87	Fixture Difficulty: 0.0
- João Pedro Junqueira de Jesus	Score: 16.99	Cost: 76	Fixture Difficulty: 0.0
Midfielders:
- Cody Gakpo	Score: 17.13	Cost: 76	Fixture Difficulty: 0.0
- Josh Cullen	Score: 17.63	Cost: 50	Fixture Difficulty: 0.0
- Antoine Semenyo (Vice Captain) 	Score: 20.5	Cost: 72	Fixture Difficulty: 0.0
- Dan Ndoye	Score: 15.83	Cost: 60	Fixture Difficulty: 0.0


In [40]:
new_team_cost = selected_team.getTotalCost()
new_team_cost

added_cost = new_team_cost - current_team_cost
print("Added cost:",added_cost)

surplus = current_team_cost - new_team_cost


Added cost: -36


In [41]:
#HEURISTIC = "combined"
#MODE = SolverMode.CHEAPEST_FIRST

In [42]:
# all_player_data["score"] = all_player_data[HEURISTIC] * all_player_data["form"]
deviation = np.std(all_player_data["score"])
scale_factor = deviation
scale_factor, deviation

(4.193334086968382, 4.193334086968382)

In [43]:
START_SAMPLE_GAMEWEEK = config.CURRENT_GAMEWEEK
END_SAMPLE_GAMEWEEK = min(config.CURRENT_GAMEWEEK+2, 39)

In [44]:
matrix = FixtureDifficultyMatrix(scale_factor, START_SAMPLE_GAMEWEEK, END_SAMPLE_GAMEWEEK, config.CURRENT_SEASON)
matrix.precomputeFixtureDifficulty(scale_factor)

new_players.recalculateFixtureDifficulty(matrix)
# new_players.calculateScore(HEURISTIC)

# selected_team.recalculateFixtureDifficulty(matrix)
# selected_team.calculateScore(HEURISTIC)

# current_team.recalculateFixtureDifficulty(matrix)
# current_team.calculateScore(HEURISTIC)

In [45]:
print("Current Team:")
display(current_team)
print()
print("Selected Team:")
display(selected_team)

Current Team:



Selected Team:


In [46]:
score_dif = selected_team.getTotalScore() - current_team.getTotalScore()
print("Score difference:",score_dif)

Score difference: 130.26999999999998


In [47]:
def getBest(pCurrentTeam: Team, pNewTeam: Team, pNewPlayers: Team, pPosition: Position):
    currentPositionData = pCurrentTeam.getPlayersListByPosition(pPosition)
    newPlayersPositionData = pNewPlayers.getPlayersListByPosition(pPosition)
    oldTotalCost = pCurrentTeam.getTotalCost()
    maxCost = max(oldTotalCost, TOTAL_BUDGET)
    #maxCost = TOTAL_BUDGET

    allTransfers = []

    for i in range(len(currentPositionData)):
        oldPlayer = currentPositionData[i]
        oldPlayerCost = oldPlayer.getCost()
        for j in range(len(newPlayersPositionData)):

            newPlayer = newPlayersPositionData[j]
            newCost = oldTotalCost - oldPlayerCost + newPlayer.getCost()

            if (newCost <= maxCost):
                allTransfers.append(Transfer(oldPlayer, newPlayer))

    if (len(allTransfers) >= 1):
        bestTransfer = max(allTransfers)
        if(bestTransfer.getScoreDif() > 0):
            return bestTransfer
    else:
        return None

In [48]:
def get_updated_team(team: pd.DataFrame, old_player: pd.Series, new_player: pd.Series):
    team: pd.DataFrame = team.drop(index=old_player.name)
    team.loc[len(team)] = new_player
    return team

In [49]:
def get_bench(team: pd.DataFrame):
    positions = ["FWD","DEF","MID","GKP"]
    team = team.reset_index()
    bench = pd.DataFrame(columns=team.columns)
    for position in positions:
        worst_player_index = team.loc[team["position"]==position]["score"].idxmin()
        worst_player = team.loc[worst_player_index].copy()
        bench.loc[len(bench)] = worst_player
        team = team.drop(index=worst_player_index)
    return team, bench

In [50]:
def getBestTransferNew(pCurrentTeam: Team, pNewTeam: Team, pNewPlayers: Team) -> Transfer | None:
    positions = Position.listValues()
    bestTransfers = []
    for position in positions:
        transfer = getBest(pCurrentTeam, pNewTeam, pNewPlayers, position)
        if(transfer is not None):
            bestTransfers.append(transfer)
    actualBestTransfer = max(bestTransfers)
    return actualBestTransfer

In [51]:
def getNewTeam(pCurrentTeam: Team, pSelectedTeam: Team, pNewPlayers: Team, pMatrix: FixtureDifficultyMatrix):

    TRIPLE_CAPT_THRESHOLD = 0.1

    transferData = getBestTransferNew(pCurrentTeam, pSelectedTeam, pNewPlayers)
    print("Best transfer:")
    print(transferData)
    oldPlayer = transferData.getOldPlayer()
    newPlayer = transferData.getNewPlayer()
    if transferData is None:
        return
    
    changingPosition: Position = transferData.getPosition()
    
    newTeam = deepcopy(pCurrentTeam)
    playersOfPosition = newTeam.getPlayersListByPosition(changingPosition)

    for i in range(len(playersOfPosition)):
        player = playersOfPosition[i]
        if (player.getId() == oldPlayer.getId()):
            newTeam.removePlayerByIndex(i, changingPosition)
            newTeam.addPlayer(newPlayer)
    

    newTeam.recalculateFixtureDifficulty(pMatrix)

    for player in newTeam.getPlayers():
        if(player.getCurrentDifficulty() < TRIPLE_CAPT_THRESHOLD):
            print(f"Suggested triple captain: {player.getName()}")
    return newTeam

In [52]:
newTeam = getNewTeam(current_team, selected_team, new_players, matrix)
newTeam = newTeam.toBenchTeam()
display(newTeam)

Best transfer:
Transfer from Yoane Wissa -> Hugo Ekitiké:
Old player: Yoane Wissa	Score: 0.0	Cost: 75	Fixture Difficulty: 0.0
New player: Hugo Ekitiké	Score: 19.67	Cost: 87	Fixture Difficulty: -1.5497104234448371
- Cost change: 12
- Score change: 19.67
